<a href="https://colab.research.google.com/github/thecolorblue/Youtube-Comment-Chatbot/blob/main/create_channel_file.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install progress
!pip install openai

In [1]:
import os
import google_auth_oauthlib.flow
import googleapiclient.discovery
import googleapiclient.errors
import requests

# Replace with your own API key
# instructions to get your key are here:
# https://developers.google.com/youtube/v3/getting-started
yt_api_key = "YOUR_API_KEY"

# replace with the channel id (not the username)
# lookup the channel id here:
# https://developers.google.com/youtube/v3/docs/channels/list
channel_id = 'UCJ0FT6Ijeyf6Mr-GdOBsVAQ'

def get_video_ids(channel_id):
    videos = []
    base_url = "https://www.googleapis.com/youtube/v3/"
    first_url = f"{base_url}search?key={yt_api_key}&channelId={channel_id}&part=id,snippet&order=date&maxResults=100"
    url = first_url

    while True:
        response = requests.get(url)
        data = response.json()
        if 'error' in data:
          print(data['error'])
          break

        if 'items' in data:
          for item in data["items"]:
              if item["id"]["kind"] == "youtube#video":
                videos.append((
                    item["id"]["videoId"],
                    item["snippet"]["title"],
                    item["snippet"]["description"],
                ))

        try:
            next_token = data["nextPageToken"]
            url = f"{first_url}&pageToken={next_token}"
        except KeyError:
            break

    return videos


def get_comments(video):
    video_id, title, description = video
    comments = []
    base_url = "https://www.googleapis.com/youtube/v3/"
    first_url = f"{base_url}commentThreads?key={yt_api_key}&videoId={video_id}&part=snippet&maxResults=100"

    url = first_url
    while True:
        response = requests.get(url)
        data = response.json()
        if 'error' in data:
          print(data['error'])
          break

        for item in data["items"]:
            comment = item["snippet"]["topLevelComment"]["snippet"]["textOriginal"]
            comments.append(comment)

        try:
            next_token = data["nextPageToken"]
            url = f"{first_url}&pageToken={next_token}"
        except KeyError:
            break

    return comments




In [ ]:
from tqdm import tqdm
import pandas as pd

video_ids = get_video_ids(channel_id)

all_comments = []
print(video_ids)
for video in tqdm(video_ids):
    comments = get_comments(video)
    all_comments.append(comments)

print('comments found:')
print(len([comment for video_comments in all_comments for comment in video_comments]))

df = pd.DataFrame({
    'comments': all_comments,
    'video_id': [i[0] for i in video_ids],
    'title': [i[1] for i in video_ids],
    'description': [i[2] for i in video_ids]
})

file_path = f"./drive/MyDrive/yt_channels/channel_{channel_id}.feather"
df.to_feather(file_path)

In [ ]:
!pip install openai
!pip install progress

In [ ]:
import openai
import pandas as pd
from progress.bar import IncrementalBar
import concurrent.futures

EMBEDDING_MODEL = "text-embedding-ada-002"
GPT_MODEL = "gpt-3.5-turbo"

# Open AI API key
# You can create one here:
# https://platform.openai.com/account/api-keys
OPENAI_API_KEY = ''
file_name = f'channel_{channel_id}'

# the channel files will look like this:
# pd.DataFrame({
#     'comments': all_comments,
#     'video_id': [i[0] for i in video_ids],
#     'title': [i[1] for i in video_ids],
#     'description': [i[2] for i in video_ids]
# })

channel = pd.concat([
  pd.read_feather(f'/content/drive/MyDrive/yt_channels/{file_name}.feather'),
])

sentences = [comment for video_comments in channel['comments'] for comment in video_comments]

print(len(sentences))

In [ ]:

from tqdm import tqdm

results = []
df = pd.DataFrame()

for sentence in tqdm(sentences, mininterval=15, miniters=len(sentences)/100):
    try:
      response = openai.Embedding.create(api_key=OPENAI_API_KEY,
                      input=[str(sentence)[0:8190]],
                      model=EMBEDDING_MODEL)
    except Exception as e:
      print(e)
      print(sentences)
      continue

    embedding = response["data"][0]["embedding"]
    results.append({"quote": sentence, "embedding": embedding})

df = pd.DataFrame(results)

df.to_feather('./drive/MyDrive/yt_comments/scottish_murmurs_comments.feather')